In [1]:
# default_exp datasets

# Lohnas2014 Dataset
> Siegel, L. L., & Kahana, M. J. (2014). A retrieved context account of spacing and repetition effects in free recall. Journal of Experimental Psychology: Learning, Memory, and Cognition, 40(3), 755.

Across 4 sessions, 35 subjects performed delayed free recall of 48 lists. Subjects were University of Pennsylvania undergraduates, graduates and staff, age 18-32. List items were drawn from a pool of 1638 words taken from the University of South Florida free association norms (Nelson, McEvoy, & Schreiber, 2004; Steyvers, Shiffrin, & Nelson, 2004, available at http://memory.psych.upenn.edu/files/wordpools/PEERS_wordpool.zip). Within each session, words were drawn without replacement. Words could repeat across sessions so long as they did not repeat in two successive sessions. Words were also selected to ensure that no strong semantic associates co-occurred in a given list (i.e., the semantic relatedness between any two words on a given list, as determined using WAS (Steyvers et al., 2004), did not exceed a threshold value of 0.55).

Subjects encountered four different types of lists: 
1. Control lists that contained all once-presented items;  
2. pure massed lists containing all twice-presented items; 
3. pure spaced lists consisting of items presented twice at lags 1-8, where lag is defined as the number of intervening items between a repeated item's presentations; 
4. mixed lists consisting of once presented, massed and spaced items. Within each session, subjects encountered three lists of each of these four types. 

In each list there were 40 presentation positions, such that in the control lists each position was occupied by a unique list item, and in the pure massed and pure spaced lists, 20 unique words were presented twice to occupy the 40 positions. In the mixed lists 28 once-presented and six twice-presented words occupied the 40 positions. In the pure spaced lists, spacings of repeated items were chosen so that each of the lags 1-8 occurred with equal probability. In the mixed lists, massed repetitions (lag=0) and spaced repetitions (lags 1-8) were chosen such that each of the 9 lags of 0-8 were used exactly twice within each session. The order of presentation for the different list types was randomized within each session. For the first session, the first four lists were chosen so that each list type was presented exactly once. An experimenter sat in with the subject for these first four lists, though no subject had difficulty understanding the task.

The data for this experiment is stored in `data/repFR.mat`. We define a unique `prepare_lohnas2014_data` function to build structures from the dataset that works with our existing data analysis and fitting functions.

Like in `prepare_murdock1962_data`, we need list lengths, a data frame for visualizations with psifir, and a trials array encoding recall events as sequences of presentation positions. But we'll also need an additional array tracking presentation order, too.

In [2]:
from compmemlearn.datasets import find_first

In [3]:
# export

import scipy.io as sio
import numpy as np
import pandas as pd
from psifr import fr


def prepare_lohnas2014_data(path):
    """
    Prepares data formatted like `data/repFR.mat` for fitting.
    """
    
    # load all the data
    matfile = sio.loadmat(path, squeeze_me=True)['data'].item()
    subjects = matfile[0]
    pres_itemnos = matfile[4]
    recalls = matfile[6]
    list_types = matfile[7]
    list_length = matfile[12]
    
    # convert pres_itemnos into rows of unique indices for easier model encoding
    presentations = []
    for i in range(len(pres_itemnos)):
        seen = []
        presentations.append([])
        for p in pres_itemnos[i]:
            if p not in seen:
                seen.append(p)
            presentations[-1].append(seen.index(p))
    presentations = np.array(presentations)

    # discard intrusions from recalls
    trials = []
    for i in range(len(recalls)):
        trials.append([])
        
        trial = list(recalls[i])
        for t in trial:
            if (t > 0) and (t not in trials[-1]):
                trials[-1].append(t)
        
        while len(trials[-1]) < list_length:
            trials[-1].append(0)
            
    trials = np.array(trials)
    
    # encode dataset into psifr format
    data = []
    for trial_index, trial in enumerate(trials):
        presentation = presentations[trial_index]
        
        # every time the subject changes, reset list_index
        if not data or data[-1][0] != subjects[trial_index]:
            list_index = 0
        list_index += 1
        
        # add study events
        for presentation_index, presentation_event in enumerate(presentation):
            data += [[subjects[trial_index], 
                      list_index, 'study', presentation_index+1, presentation_event, list_types[trial_index], 
                      find_first(presentation_event, presentation) + 1
                     ]]
            
        # add recall events
        for recall_index, recall_event in enumerate(trial):
            if recall_event != 0:
                data += [[subjects[trial_index], list_index, 
                          'recall', recall_index+1, presentation[recall_event-1], list_types[trial_index], recall_event
                         ]]
                
    data = pd.DataFrame(data, columns=[
        'subject', 'list', 'trial_type', 'position', 'item', 'condition', 'first_input'])
    merged = fr.merge_free_recall(data, list_keys=['condition', 'first_input'])
    
    return trials, merged, list_length, presentations, list_types, data, subjects

In [4]:
trials, events, list_length, presentations, list_types, rep_data, subjects = prepare_lohnas2014_data(
    '../../data/repFR.mat')

events.loc[(events.subject==1) & (events.list==1)]

,subject,list,item,input,output,study,recall,repeat,intrusion,condition,first_input
0,1,1,0,1,1.0,True,True,0,False,4,1
1,1,1,1,2,2.0,True,True,0,False,4,2
2,1,1,2,3,3.0,True,True,0,False,4,3
3,1,1,3,4,4.0,True,True,0,False,4,4
4,1,1,4,5,5.0,True,True,0,False,4,5
5,1,1,5,6,6.0,True,True,0,False,4,6
6,1,1,6,7,7.0,True,True,0,False,4,7
7,1,1,7,8,NaN,True,False,0,False,4,8
8,1,1,8,9,8.0,True,True,0,False,4,9
9,1,1,9,10,9.0,True,True,0,False,4,10


In [5]:
assert(np.sum(events.intrusion) == 0)

In [6]:
from compmemlearn.datasets import events_metadata

alt_trials, alt_list_lengths, alt_presentations = events_metadata(events)
assert(np.all(trials[:, :37] == alt_trials[0]))
assert(np.all(trials[list_types==4][:, :37]==alt_trials[0][list_types==4]))
assert(np.all(presentations[list_types==4]==alt_presentations[0][list_types==4]))

In [7]:
assert(np.all(alt_trials[0][0] == trials[0, :37]))

In [8]:
from compmemlearn.datasets import generate_trial_mask
assert(np.all(generate_trial_mask(events, 'condition == 4')[0] == (list_types==4)))
assert(np.all(trials[generate_trial_mask(events, 'condition == 4')[0]] == trials[list_types==4]))

In [9]:
#slow

events.to_csv('../../data/LohnasKahana2014.csv', index=False)

In [11]:
pd.read_csv('../../data/LohnasKahana2014.csv').loc[(events.subject==1) & (events.list==1)]

,subject,list,item,input,output,study,recall,repeat,intrusion,condition,first_input
0,1,1,0,1,1.0,True,True,0,False,4,1
1,1,1,1,2,2.0,True,True,0,False,4,2
2,1,1,2,3,3.0,True,True,0,False,4,3
3,1,1,3,4,4.0,True,True,0,False,4,4
4,1,1,4,5,5.0,True,True,0,False,4,5
5,1,1,5,6,6.0,True,True,0,False,4,6
6,1,1,6,7,7.0,True,True,0,False,4,7
7,1,1,7,8,NaN,True,False,0,False,4,8
8,1,1,8,9,8.0,True,True,0,False,4,9
9,1,1,9,10,9.0,True,True,0,False,4,10
